In [1]:
import os
os.chdir('../')

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    model_path: str
    test_name: str
    data_file_path: Path
    params_epochs: int
    params_batch_size: int


In [3]:
from HowlerMonkey.constants import *
from HowlerMonkey.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self):

        training = self.config.training

        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            model_path = training.model_path,
            data_file_path = Path(training.data_file_path),
            params_epochs = self.params.EPOCHS,
            params_batch_size = self.params.BATCH_SIZE,
            params_image_size = self.params.IMSIZE,
            test_name = self.params.TEST_NAME
        )
        
        return training_config

In [5]:
import os
import torch
import shutil
from ultralytics import YOLO


from HowlerMonkey.utils.common import get_latest_model

/Users/guido/Library/CloudStorage/OneDrive-PUCRS-BR/Mestrado/Dissertacao/Howler-Monkey/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_model(self):
        self.model = YOLO(self.config.model_path)

    def save_model(self, path: Path):
        best_model_path = get_latest_model(self.config.root_dir)
        shutil.copy(best_model_path, path)

    
    def train(self):
            
        self.model.train(
            data=self.config.data_file_path,
            epochs=self.config.params_epochs,
            batch=self.config.params_batch_size,
            project=self.config.root_dir,
            imgsz=self.config.params_image_size,
            name=self.config.test_name
        )

        self.save_model(self.config.root_dir)


In [9]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(training_config)
    training.get_model()
    training.train()
except Exception as e:
    raise e

[2024-04-01 09:42:59,859: INFO: common] yaml file: config/config.yaml loaded successfully
[2024-04-01 09:42:59,861: INFO: common] yaml file: params.yaml loaded successfully
[2024-04-01 09:42:59,861: INFO: common] Creating directory: artifacts
[2024-04-01 09:42:59,862: INFO: common] Creating directory: artifacts/training
New https://pypi.org/project/ultralytics/8.1.40 available 😃 Update with 'pip install -U ultralytics'


SyntaxError: '[31m[1mmlflow[0m' is not a valid YOLO argument. 

    Arguments received: ['yolo', '--f=/Users/guido/Library/Jupyter/runtime/kernel-v2-936l9OR16QFifKS.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'obb', 'pose', 'detect', 'classify', 'segment'}
                MODE (required) is one of {'val', 'train', 'track', 'predict', 'benchmark', 'export'}
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco128.yaml model=yolov8n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at image size 320:
        yolo predict model=yolov8n-seg.pt source='https://youtu.be/LNwODJXcvt4' imgsz=320

